# IMPORT LIBRARIES

In [2]:
#### Check for libraries and updates ####
import sys
import subprocess
import pkg_resources

required = {"tensorflow", "scikit-learn", "h5py", "numpy", "pandas", "scipy", "keras", "scikeras[tensorflow]"}
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

print("Package installation done!")


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


Package installation done!


In [3]:
#### Import libraries ####

import h5py
import os
import numpy as np
import pandas as pd
#import scipy
from datetime import datetime
#import librosa
#import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
import random
import tensorflow as tf
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.layers import Bidirectional, LSTM, Dense, GRU, MaxPooling2D, Conv2D, Flatten, Dropout, BatchNormalization, Activation, MaxPooling1D, Conv1D, GlobalAveragePooling1D, GlobalAveragePooling2D, GlobalMaxPooling2D, GlobalMaxPooling1D
from sklearn.model_selection import StratifiedKFold, KFold
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.utils import Sequence
from scikeras.wrappers import KerasClassifier
import keras
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
print("Libraries imported!")

2023-11-16 15:34:05.030488: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-16 15:34:05.123896: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-16 15:34:05.124553: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-16 15:34:07.952712: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Libraries imported!


# IMPORT DATA

In [4]:
# Import merged data from C:\Users\Saulo Mendes Santos\OneDrive\Documents\2. LETRAS\0. Doutorado\0. Recherche\2.2. Voicing Decision Modelling\VD_model_03-context\RNN_2D-vector\20230130_Bidirectional-RNN_server-malbec

#file_path = "C:/Users/Saulo Mendes Santos/OneDrive/Documents/2. LETRAS/0. Doutorado/0. Recherche/2.2. Voicing Decision Modelling/VD_model_03-context/RNN_2D-vector/20230130_Bidirectional-RNN_server-malbec/merged_CSTR-KEELE_file.hdf5"

# Import data h5py file
file_path = '/mnt/c/Users/Saulo Mendes Santos/OneDrive/Documents/2. LETRAS/0. Doutorado/0. Recherche/1.1. F0 final/TODOS_resampled_5ms_20231115.hdf5'

In [8]:
# Define function to stack data
# We change the function to customize the number of folds
# In this specific case, we want to have just one fold for training and evaluation
# We also use the shuffle parameter to shuffle the data before splitting it into folds
def stack_split_data(f, validation_split=0.25):
    """
    Stack and fold the data from the h5py file.
    f: h5py file

    """
    # we want to pad the last 7 rows with zeros
    pad_width = ((3, 4), (0, 0))
    # the constant mode will use the edge values to fill the padded regions with zeros as default
    mode = 'constant'

    # Get the list of groups in the h5py file
    groups = list(f.keys())

    # Filter out the groups containing "_egg" and groups that do not contain "_" from the groups list
    groups = [group for group in groups if "_egg" not in group and "_" in group]

    # Create a dictionary to associate the index of the group as a key to the group name as the value
    group_dict = {i: group for i, group in enumerate(groups)}

    # Attention: instead of using KFold, we will build an array with the indices of the groups and randomly pick items for training and validation sets
    # This is necessary because KFold does not allow to have a group with only one fold: n_splits must be at least 2
    # We will use the same random state to make sure that the same groups are picked for training and validation
    ids_array = np.arange(len(groups))
    np.random.seed(42)
    np.random.shuffle(ids_array)
    training_ratio = 1 - validation_split
    train_ids = ids_array[:int(len(ids_array) * training_ratio)]
    valid_ids = ids_array[int(len(ids_array) * training_ratio):]
    train_arrays, valid_arrays = [], []

    # Now we iterate over the groups and stack the data
    for train_i in train_ids:
        group_name = group_dict[train_i]
        # Get the dataset in group
        data = np.array(f[group_name])
        # But we need to pad the edges of the 0-th axis with the same values of the first and last lines
        padded_data = np.pad(data, pad_width=pad_width, mode=mode)
        # Add the id of the group as a column of the data
        padded_data = np.hstack((padded_data, np.ones((padded_data.shape[0], 1)) * train_i))
        # Append the padded data to the list
        train_arrays.append(padded_data)
    
    if validation_split != 1:
        train_data = np.vstack(train_arrays)
    else:
        train_data = np.nan
    
    # Now we do the same but for the validation data
    for valid_i in valid_ids:
        group_name = group_dict[valid_i]
        data = np.array(f[group_name])
        padded_data = np.pad(data, pad_width=pad_width, mode=mode)
        padded_data = np.hstack((padded_data, np.ones((padded_data.shape[0], 1)) * valid_i))
        valid_arrays.append(padded_data)

    if validation_split != 0:
        valid_data = np.vstack(valid_arrays)
    else:
        valid_data = np.nan

    return train_data, valid_data, group_dict

In [62]:
f = h5py.File(file_path, 'r')
train_data, test_data, group_dict = stack_split_data(f, validation_split=1)
f.close()

In [44]:
# Check the shape of the data
#print(train_data.shape)
print(test_data.shape)

(319223, 41)


In [21]:
# Let's check last MFCC data
test_data[-10, 39]

0.16919615864753723

Since we are going to calculate Voicing Decision Error rates only of test data, we should have enough files for data

In [12]:
# Check if we have the index of the file in the last column
test_data[:,-1]

array([265., 265., 265., ..., 102., 102., 102.])

In [13]:
# Check the first 10 keys and values of the group_dict
list(group_dict.items())[:10]

[(0, 'bfamcv01__002__GIL'),
 (1, 'bfamcv01__019__EVN'),
 (2, 'bfamcv01__048__EVN'),
 (3, 'bfamcv01__051__LUI'),
 (4, 'bfamcv01__054__EVN'),
 (5, 'bfamcv01__062__GIL'),
 (6, 'bfamcv01__068__GIL'),
 (7, 'bfamcv01__127__LEO'),
 (8, 'bfamcv01__141__EVN'),
 (9, 'bfamcv01__142__GIL')]

# DEFINE MODEL

In [14]:
# Build model layers
# Define and wrap the model
# import KerasClassifier from keras wrappers
# Define the function that creates the model
def create_model(input_shape=(7, 38, 1)):
    # create model
    tf.keras.backend.clear_session()

    cnn = Sequential()
    # convolutional layer with rectified linear unit activation
    cnn.add(Conv2D(32, kernel_size=(3, 3),
                    activation='relu',
                    input_shape=input_shape))
    # 32 convolution filters used each of size 3x3
    # again
    cnn.add(Conv2D(64, (3, 3), activation='relu'))
    # 64 convolution filters used each of size 3x3
    # choose the best features via pooling
    cnn.add(MaxPooling2D(pool_size=(2, 2)))
    # randomly turn neurons on and off to improve convergence
    # TODO: double the dropout rate to 0.5 during the hyperparameter tuning
    cnn.add(Dropout(0.25))
    # flatten since too many dimensions, we only want a classification output
    cnn.add(Flatten())
    # fully connected to get all relevant data
    cnn.add(Dense(128, activation='relu'))
    # one more dropout for convergence' sake :) 
    cnn.add(Dropout(0.5))
    # output a softmax to squash the matrix into output probabilities
    cnn.add(Dense(1, activation='sigmoid'))
    # TODO: change the metrics to recall and precision: f1-score.
    cnn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return cnn

# TRAIN MODEL

In [15]:
# Define the Sliding window generator
class SlidingWindowDataGenerator(Sequence):
    def __init__(self, x, y, window_size, batch_size, shuffle=True):
        self.x = x
        self.y = y
        self.window_size = window_size
        self.batch_size = batch_size
        self.shuffle = shuffle
        # Problem: the last window
        #self.indices = np.arange(len(self.x))
        self.indices = np.arange(len(self.x)-(self.window_size[0]))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.x) / self.batch_size))

    def __getitem__(self, index):
        # O problema está aqui
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        # print("batch_indices: ", batch_indices)
        batch_x = []
        batch_y = []
        for i in batch_indices:
            # x shape: (5639045, 38)
            # window_size = (7, 38, 1)
            window = self.x[i:i+self.window_size[0],:].reshape(*self.window_size)
            # We need to the label 
            label = self.y[i + (self.window_size[0]//2)]
            batch_x.append(window)
            batch_y.append(label)
        batch_x = np.array(batch_x)
        # Reshape the batch to train a CNN
        # batch_x = batch_x.reshape(batch_x.shape[0], batch_x.shape[1], batch_x.shape[2], 1)
        # batch_y = to_categorical(np.array(batch_y, dtype=int), num_classes=np.max(self.y)+1)
        # Convert the list to an array of integers
        # batch_y = to_categorical([int(x) for x in batch_y])
        batch_y = np.array(batch_y, dtype=int)

        # print sizes
        # print('batch_x shape: {}'.format(batch_x.shape))
        # print('batch_y shape: {}'.format(batch_y.shape))

        return batch_x, batch_y

    def on_epoch_end(self):
        if self.shuffle:
            # Random state is set to 42 to ensure reproducibility
            np.random.seed(42)
            np.random.shuffle(self.indices)


In [75]:
class SlidingWindowDataGeneratorPrediction(Sequence):
    def __init__(self, x, window_size, batch_size):
        self.x = x
        self.window_size = window_size
        self.batch_size = batch_size
        self.indices = np.arange(len(self.x) - (self.window_size[0]))
        self.on_epoch_end()

    def __len__(self):
        return int(np.ceil(len(self.x) / self.batch_size))

    def __getitem__(self, index):
        batch_indices = self.indices[index * self.batch_size: (index + 1) * self.batch_size]
        batch_x = []

        for i in batch_indices:
            window = self.x[i: i + self.window_size[0], :].reshape(*self.window_size)
            batch_x.append(window)

        batch_x = np.array(batch_x)

        return batch_x

    def on_epoch_end(self):
        # No shuffling for prediction
        pass


In this case, we will have 3 models:
- One only with MFCCs
- One only with f0 estimations
- One with all the features

We will have the predictions made by these models stacked to the data matrix, which should finally be save into an hdf5 file.

In [9]:
"""
Contenu des colonnes :
0   :   timeframes (not used for the model)
1   :   creacky_voice decision (0 = no creacky voice, 1 = creacky voice) Not used for the model
2   :   bana_resampled
3   :   opensmile_resampled
4   :   pefac_resampled
5   :   praat_ac_resampled
6   :   praat_cc_resampled
7   :   praat_shs_resampled
8   :   pyin_resampled
9   :   rapt_resampled
10  :   srh_resampled
11  :   straight_resampled
12  :   swipe_resampled
13  :   swipep_resampled
14  :   yaapt_resampled
15  :   yin_resampled
16  :   hnr_resampled
17  :   cpps_resampled
18  :   praat_intensity_resampled
19  :   praat_se_resampled
20  :   mfcc_01
21  :   mfcc_02
22  :   mfcc_03
23  :   mfcc_04
24  :   mfcc_05
25  :   mfcc_06
26  :   mfcc_07
27  :   mfcc_08
28  :   mfcc_09
29  :   mfcc_10
30  :   mfcc_11
31  :   mfcc_12
32  :   mfcc_13
33  :   mfcc_14
34  :   mfcc_15
35  :   mfcc_16
36  :   mfcc_17
37  :   mfcc_18
38  :   mfcc_19
39  :   mfcc_20
40  :   group_index / file_index (Not used for the model)

"""

'\nContenu des colonnes :\n0   :   ground_truth\n1   :   voiced_ground_truth\n2   :   bana_resampled\n3   :   opensmile_resampled\n4   :   pefac_resampled\n5   :   praat_ac_resampled\n6   :   praat_cc_resampled\n7   :   praat_shs_resampled\n8   :   pyin_resampled\n9   :   rapt_resampled\n10  :   srh_resampled\n11  :   straight_resampled\n12  :   swipe_resampled\n13  :   swipep_resampled\n14  :   yaapt_resampled\n15  :   yin_resampled\n16  :   hnr_resampled\n17  :   cpps_resampled\n18  :   praat_intensity_resampled\n19  :   praat_se_resampled\n20  :   mfcc_01\n21  :   mfcc_02\n22  :   mfcc_03\n23  :   mfcc_04\n24  :   mfcc_05\n25  :   mfcc_06\n26  :   mfcc_07\n27  :   mfcc_08\n28  :   mfcc_09\n29  :   mfcc_10\n30  :   mfcc_11\n31  :   mfcc_12\n32  :   mfcc_13\n33  :   mfcc_14\n34  :   mfcc_15\n35  :   mfcc_16\n36  :   mfcc_17\n37  :   mfcc_18\n38  :   mfcc_19\n39  :   mfcc_20\n40  :   group_index / file_index\n\n'

In [34]:
# Windows sizes for the different models
window_size_all = (7, 38, 1)
window_size_mfcc = (7, 20, 1)
window_size_f0 = (7, 18, 1)
window_size_list = [window_size_all, window_size_mfcc, window_size_f0]

# We initialize the models and the indices that will be used to train and test the models
cnn_all = create_model(input_shape=window_size_all)
cnn_mfcc = create_model(input_shape=window_size_mfcc)
cnn_f0 = create_model(input_shape=window_size_f0)
models = [cnn_all, cnn_mfcc, cnn_f0]

# Indices for the different models
indices_all = (2, 40)
indices_mfcc = (20, 40)
indices_f0 = (2, 20)
indices_list = [indices_all, indices_mfcc, indices_f0]


# Iterate over the folds and train/test sets
histories = []
model_names = ['cnn_all', 'cnn_mfcc', 'cnn_f0']

In [16]:
# This part is not used since we are predicting new values stored in the test_data array
"""
batch_size = 256

# We train each model according to the indices
for i in range(len(models)):
    print("Training model: ", model_names[i])

    # We define the training and testing sets according to the indices
    x_train, y_train = train_data[:, indices_list[i][0]:indices_list[i][1]], train_data[:, 1]
    x_test, y_test = test_data[:, indices_list[i][0]:indices_list[i][1]], test_data[:, 1]
    
    # We define the generators
    train_generator = SlidingWindowDataGenerator(x_train, y_train, window_size_list[i], batch_size)
    valid_generator = SlidingWindowDataGenerator(x_test, y_test, window_size_list[i], batch_size)

    # We train the model
    history = models[i].fit(train_generator, validation_data=valid_generator, epochs=10, verbose=1)
    histories.append(history)

    # We save the model
    models[i].save(model_names[i] + '.h5')

    # We save the history
    with open(model_names[i] + '_history.pkl', 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

    # We predict the test set and stack it vertically to the test_data array
    # We use the slinding window generator to predict the test set, but we do not shuffle the data
    test_generator_pred = SlidingWindowDataGenerator(x_test, y_test, window_size_list[i], batch_size, shuffle=False)
    y_pred = models[i].predict(test_generator_pred, verbose=1)

    # The sliding window generator stops 7 frames before the end of the test set, so we add 7 zeros to the end of the prediction
    # Remember that the train and test data is padded anyway
    y_pred = np.vstack((y_pred, np.zeros((7, y_pred.shape[1]))))

    # Round the predictions
    y_pred = np.round(y_pred)
    
    # Notice that since we are using the indices, the test_data used to train the next model will not be different
    test_data = np.hstack((test_data, y_pred))
"""

Training model:  cnn_all
Epoch 1/10
16313/16313 [==============================] - 1922s 118ms/step - loss: 0.3448 - accuracy: 0.8433 - val_loss: 0.3067 - val_accuracy: 0.8574
Epoch 2/10
16313/16313 [==============================] - 1696s 104ms/step - loss: 0.3136 - accuracy: 0.8549 - val_loss: 0.2949 - val_accuracy: 0.8618
Epoch 3/10
16313/16313 [==============================] - 1610s 99ms/step - loss: 0.3056 - accuracy: 0.8585 - val_loss: 0.2884 - val_accuracy: 0.8653
Epoch 4/10
16313/16313 [==============================] - 1566s 96ms/step - loss: 0.3010 - accuracy: 0.8606 - val_loss: 0.2837 - val_accuracy: 0.8665
Epoch 5/10
16313/16313 [==============================] - 1547s 95ms/step - loss: 0.2978 - accuracy: 0.8619 - val_loss: 0.2810 - val_accuracy: 0.8690
Epoch 6/10
16313/16313 [==============================] - 1594s 98ms/step - loss: 0.2953 - accuracy: 0.8631 - val_loss: 0.2767 - val_accuracy: 0.8715
Epoch 7/10
16313/16313 [==============================] - 1572s 96ms/step

# We only use the following code if we want to reload the model to do predictions

In [77]:
# import pickle
import pickle
from keras.models import load_model
batch_size = 256

# Reaload each model and do the predictions
for i in range(len(models)):
    print("Reloading model: ", model_names[i])

    # Clear previous session
    from keras import backend as K

    # We reload the model
    models[i] = load_model(model_names[i] + '.h5')

    # We reload the model
    models[i] = load_model(model_names[i] + '.h5')

    # We define the training and testing sets according to the indices
    x_test, _ = test_data[:, indices_list[i][0]:indices_list[i][1]], test_data[:, 1]

    # We predict the test set and stack it vertically to the test_data array
    # We use the slinding window generator to predict the test set, but we do not shuffle the data

    pred_generator = SlidingWindowDataGeneratorPrediction(x_test, window_size_list[i], batch_size)
    y_pred = models[i].predict(pred_generator, verbose=1)

    # The sliding window generator stops 7 frames before the end of the test set
    # Also, the first prediction is done for index 3, so we add 3 rows of zeros to the beginning of the prediction and 4 rows of zeros to the end
    # Add 3 zeros to the beginning of the prediction
    y_pred = np.vstack((np.zeros((3, y_pred.shape[1])), y_pred))
    # Add 4 zeros to the end of the prediction
    y_pred = np.vstack((y_pred, np.zeros((4, y_pred.shape[1]))))

    # Round the predictions
    y_pred = np.round(y_pred)
    
    # Notice that since we are using the indices, the test_data used to train the next model will not be different
    if i == 0:
        test_data_predicted = np.hstack((test_data, y_pred))
    else:
        test_data_predicted = np.hstack((test_data_predicted, y_pred))

Reloading model:  cnn_all


2023-11-16 19:32:12.122298: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1247/1247 [==============================] - 21s 16ms/step
Reloading model:  cnn_mfcc
   5/1247 [..............................] - ETA: 15s 

2023-11-16 19:32:33.627971: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1247/1247 [==============================] - 13s 11ms/step
Reloading model:  cnn_f0
   7/1247 [..............................] - ETA: 11s 

2023-11-16 19:32:47.905921: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


1247/1247 [==============================] - 14s 11ms/step


In [64]:
x_test.shape

(319223, 18)

In [65]:
# Check the shape of the test_data
test_data.shape

(319223, 41)

In [78]:
# Check the unique values qe have in the last three columns with a for loop and print
for i in range(3):
    print("Unique values in column {}: {}".format(i+41, np.unique(test_data_predicted[:,i+41])))
    # We need also to check the number of 1s and 0s, and nan values
    print("Number of 1s in column {}: {}".format(i+41, np.sum(test_data_predicted[:,i+41]==1)))
    print("Number of 0s in column {}: {}".format(i+41, np.sum(test_data_predicted[:,i+41]==0)))
    print("Number of nans in column {}: {}".format(i+41, np.sum(np.isnan(test_data_predicted[:,i+41]))))


Unique values in column 41: [ 0.  1. nan]
Number of 1s in column 41: 227975
Number of 0s in column 41: 90247
Number of nans in column 41: 1001
Unique values in column 42: [0. 1.]
Number of 1s in column 42: 164085
Number of 0s in column 42: 155138
Number of nans in column 42: 0
Unique values in column 43: [ 0.  1. nan]
Number of 1s in column 43: 245428
Number of 0s in column 43: 72794
Number of nans in column 43: 1001


# Rebuild the test_data in hdf5 file

"""
Contenu des colonnes :
0   :   timeframes
1   :   creaky_voice_decision
2   :   bana_resampled
3   :   opensmile_resampled
4   :   pefac_resampled
5   :   praat_ac_resampled
6   :   praat_cc_resampled
7   :   praat_shs_resampled
8   :   pyin_resampled
9   :   rapt_resampled
10  :   srh_resampled
11  :   straight_resampled
12  :   swipe_resampled
13  :   swipep_resampled
14  :   yaapt_resampled
15  :   yin_resampled
16  :   hnr_resampled
17  :   cpps_resampled
18  :   praat_intensity_resampled
19  :   praat_se_resampled
20  :   mfcc_01
21  :   mfcc_02
22  :   mfcc_03
23  :   mfcc_04
24  :   mfcc_05
25  :   mfcc_06
26  :   mfcc_07
27  :   mfcc_08
28  :   mfcc_09
29  :   mfcc_10
30  :   mfcc_11
31  :   mfcc_12
32  :   mfcc_13
33  :   mfcc_14
34  :   mfcc_15
35  :   mfcc_16
36  :   mfcc_17
37  :   mfcc_18
38  :   mfcc_19
39  :   mfcc_20
40  :   group_index / file_index
41  :   model_pred_all
42  :   model_pred_mfcc
43  :   model_pred_f0

"""

In [32]:
# Check shape of the test_data_test array
print(test_data.shape)

(1371551, 44)


In [67]:
# Check where the indices are
test_data[:, 40]

array([265., 265., 265., ..., 102., 102., 102.])

In [72]:
# We want to rebuild the the hdf5 file with the dataset and the predictions
# We first create the hdf5 file
hdf5_file = h5py.File('/mnt/c/Users/Saulo Mendes Santos/OneDrive/Documents/2. LETRAS/0. Doutorado/0. Recherche/1.1. F0 final/TODOS_resampled_5ms_VD-predictions_20231116.hdf5', 'w')

# Each group in the hdf5 file will be a file. For that we will use the group_index column(40-th index) and the group_dict dictionary alrealdy defined
# We iterate over the test_data_test array and create the groups

# We have to subset the test_data array based on the repeating indices in the 40-th column:
# We get the unique indices
unique_indices = np.unique(test_data_predicted[:, 40])
# We iterate over the unique indices and subset the test_data array
for i in range(len(unique_indices)):
    # We subset the test_data array
    test_data_subset = test_data_predicted[test_data_predicted[:, 40] == unique_indices[i]]

    # We need to round up values in column 1 to 0 or 1
    test_data_subset[:, 1] = np.round(test_data_subset[:, 1])

    # Before saving the data, we need to unpad each subset by cutting off:
    # 1. The first 3 rows
    test_data_subset = test_data_subset[3:, :]
    # 2. The last 4 rows
    test_data_subset = test_data_subset[:-4, :]
    # We get the group name in the group_dict dictionary
    group_name = group_dict[unique_indices[i]]
    # We create the group
    group = hdf5_file.create_dataset(group_name, data=test_data_subset)

# We close the hdf5 file
hdf5_file.close()

In [73]:
# We check that the groups are created properly by opening the hdf5 file with h5py.File('test_data_keele-cst.hdf5', 'r')
# We can also check the shape of each group by using the .shape attribute
# Open the hdf5 file
hdf5_file = h5py.File('/mnt/c/Users/Saulo Mendes Santos/OneDrive/Documents/2. LETRAS/0. Doutorado/0. Recherche/1.1. F0 final/TODOS_resampled_5ms_VD-predictions_20231116.hdf5', 'r')

# Get group keys
group_keys = list(hdf5_file.keys())
# Iterate over the group keys and print the shape of each group
for i in range(len(group_keys)):
    print(group_keys[i], hdf5_file[group_keys[i]].shape)

# Close the hdf5 file
hdf5_file.close()

bfamcv01__002__GIL (1846, 44)
bfamcv01__019__EVN (479, 44)
bfamcv01__048__EVN (621, 44)
bfamcv01__051__LUI (427, 44)
bfamcv01__054__EVN (593, 44)
bfamcv01__062__GIL (365, 44)
bfamcv01__068__GIL (325, 44)
bfamcv01__127__LEO (1641, 44)
bfamcv01__141__EVN (554, 44)
bfamcv01__142__GIL (303, 44)
bfamcv01__149__LUI (2714, 44)
bfamcv01__186__EVN (504, 44)
bfamcv01__239__EVN (535, 44)
bfamcv02_234_274631_0_276705_0_NOSSA (535, 44)
bfamcv02__043__RUT (506, 44)
bfamcv02__044__TER (1073, 44)
bfamcv02__045__RUT (495, 44)
bfamcv02__063__RUT (444, 44)
bfamcv02__092__RUT (388, 44)
bfamcv02__096__TER (577, 44)
bfamcv02__141__TER (468, 44)
bfamcv02__162__RUT (297, 44)
bfamcv02__178__TER (690, 44)
bfamcv02__192__RUT (952, 44)
bfamcv02__193__RUT (689, 44)
bfamcv02__202__RUT (1282, 44)
bfamcv02__224__RUT (532, 44)
bfamcv02__231__TER (474, 44)
bfamcv02__246__TER (1196, 44)
bfamcv02__264__TER (428, 44)
bfamcv02__304__TER (456, 44)
bfamcv02__315__RUT (439, 44)
bfamcv02__317__TER (589, 44)
bfamcv03__001__CEL 

In [74]:
# We want to test if file f1nw0000_16kHz_ImpResp_Classroom_+10dB has the same shape as in the raw data file
# We open the raw data file
# Import data h5py file
file_path = '/mnt/c/Users/Saulo Mendes Santos/OneDrive/Documents/2. LETRAS/0. Doutorado/0. Recherche/1.1. F0 final/TODOS_resampled_5ms_20231115.hdf5'
hdf5_file = h5py.File(file_path, 'r')
# Get the data set for file bpubmn01__098__SHE
test_size_raw = hdf5_file['bpubmn01__098__SHE'][:]
# Print shape
print(test_size_raw.shape)
# And we print the same shape for the test_data array
hdf5_file = h5py.File('/mnt/c/Users/Saulo Mendes Santos/OneDrive/Documents/2. LETRAS/0. Doutorado/0. Recherche/1.1. F0 final/TODOS_resampled_5ms_VD-predictions_20231116.hdf5', 'r')
# Get the data set for file bpubmn01__098__SHE
test_size = hdf5_file['bpubmn01__098__SHE'][:]
# Print shape
print(test_size.shape)
# Print if shapes are the same
if test_size_raw.shape[0] == test_size.shape[0]:
    print("We're good to go!")
else:
    print("Something is wrong!")
# Close the hdf5 file
hdf5_file.close()

(454, 40)
(454, 44)
We're good to go!
